# Components

In [68]:
import os
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

d = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(d)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [69]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    # base_url="https://api.llama-api.com/"
)

def ask_gpt(prompt: str, system: str, model="gpt-4o") -> str:
    response = client.chat.completions.create(
        model=model,
        # model="llama3.2-3b",
        # model="llama3.3-70b",
        messages=[
            {
                "role": "system", 
                "content": system
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        # max_tokens=512
    )
    return response.choices[0].message.content

sys_res = "Read the context, patient record, and answer the task question. If you don't know, just say you don't know."

sys_key = "Extract as much keywords and keyphrases most related to the task from the record as possible, but concisely and raw text."

In [70]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Path to the saved vector store
faiss_index_path = "/home/ngjabach/Documents/State-of-the-Art-Papers/SurgeryLLM (VIET + DONE)/faiss_index"
vector_store = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [71]:
prompt_res = '''
---
{reference}
---
A. Record: {record} 
B. Task: {task}
C. Answer:
'''

prompt_key = '''
A. Record: {record}
B. Task: {task}
'''

task1 = "Check the record and identify results outside of reference ranges."
task2 = "Identify unavailable preoperative tests."
task3 = "Surgical recommendation."
task4 = "Prepare sample operative notes."

record = '''
"Patient: Alice Johnson  
Age: 46, Female  
Medical History: Hypertension; Hyperlipidemia  
Vital Signs: BP 150/95 mmHg, HR 75 bpm  
Laboratory Findings:  
- Sodium: 148 mEq/L  
- Potassium: 3.0 mEq/L  
- Bicarbonate: 30 mEq/L  
- Creatinine: 1.8 mg/dL  
- Glucose (Fasting): 110 mg/dL  
- LDL Cholesterol: 165 mg/dL  
- Platelet count: 410,000/mm3  
- Hemoglobin A1c: 7%  
- TSH: 5.5 μU/mL  

Preoperative Workup: Comprehensive metabolic panel, lipid profile  "
'''

# print("A. Record: ", record)
# print("B. Task: ", task1)
# print("C. Answer:...")

In [72]:
class State(TypedDict):
    task: str #
    record: str #
    reference: List[Document]
    answer: str

def retrieve(state: State):
    messages = prompt_key.format(record=state["record"], task=state["task"])
    keywords = ask_gpt(messages, sys_key)
    docs = retriever.invoke(keywords)
    return {"reference": docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["reference"])
    messages = prompt_res.format(record = state["record"], task = state["task"], reference = docs_content)
    response = ask_gpt(messages, sys_res)
    return {"answer": response}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [73]:
response = graph.invoke({"task": task1, "record": record})
print(response["answer"])

Based on the patient record and reference ranges provided, the following results are outside of the reference ranges:

1. **Sodium**: 148 mEq/L (Reference range: 136-145 mEq/L) - Elevated
2. **Potassium**: 3.0 mEq/L (Reference range: 3.5-5.0 mEq/L) - Low
3. **Bicarbonate**: 30 mEq/L (Reference range: 22-28 mEq/L) - Elevated
4. **Creatinine**: 1.8 mg/dL (Reference range: 0.6-1.2 mg/dL) - Elevated
5. **Glucose (Fasting)**: 110 mg/dL (Reference range: 70-100 mg/dL) - Elevated
6. **LDL Cholesterol**: 165 mg/dL (Note: Typically, a desirable level is less than 100 mg/dL, though this can vary with guidelines) - Elevated
7. **Hemoglobin A1c**: 7% (Reference range: ≤6%) - Elevated
8. **TSH**: 5.5 μU/mL (Normal range is typically around 0.4-4.0 μU/mL, though this can vary with specific guidelines) - Elevated

These values indicate abnormalities related to electrolyte imbalance, renal function, glucose control, and lipid levels.
